In [ ]:
import sys
import torchvision
import transformers
import numpy as np
import torch

print(sys.version)
print('numpy:', np.__version__)
print('torch:',torch.__version__)
print('transformers:', transformers.__version__)
print(f"torchvision version: {torchvision.__version__}")

d:\Utilities\miniconda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.12.12 | packaged by Anaconda, Inc. | (main, Oct 14 2025, 16:10:16) [MSC v.1929 64 bit (AMD64)]
numpy: 2.2.6
torch: 2.7.0+cu128
transformers: 4.57.1
torchvision version: 0.22.0+cu128


In [4]:
import flash_attn
print(flash_attn.__version__)

2.7.4.post1


In [5]:
torch.cuda.is_available()

True

In [ ]:
import os
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

def load_ocr_model():
    # ✅ Optimized 4-bit quantization config for Blackwell GPUs
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,     # Nested quantization → less VRAM
        bnb_4bit_quant_type="nf4",          # Best quantization format for LLMs
        bnb_4bit_compute_dtype=torch.bfloat16,  # BF16 is optimal on Blackwell
    )

    os.environ['TRANSFORMERS_VERBOSITY'] = 'info'  # see Output more detail
    MODEL_PATH = "nanonets/Nanonets-OCR2-3B"


    model = AutoModelForImageTextToText.from_pretrained(
        MODEL_PATH,
        quantization_config=bnb_config,
        dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation="flash_attention_2",
    )

    torch.backends.cuda.matmul.allow_tf32 = True     # TensorFloat-32 acceleration
    torch.backends.cudnn.benchmark = True            # Optimize kernel selection
    torch.set_float32_matmul_precision("high")
    model.eval()

    processor = AutoProcessor.from_pretrained(MODEL_PATH) #? also act as Tokenizer

    return model, processor

def ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=4096):
    # prompt = """
    #     Extract the text from the above document as if you were reading it naturally.
    #     Return the tables in html format. Return the equations in LaTeX representation.
    #     If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>.
    #     Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>.
    #     Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>.
    #     Prefer using ☐ and ☑ for check boxes."""

    system_prompt = """
        Hãy trích xuất toàn bộ văn bản từ tài liệu ở trên giống như cách bạn đọc nó một cách tự nhiên.
        Trả về các bảng dưới dạng mã HTML.
        Trả về các phương trình dưới dạng biểu diễn LaTeX.

        Nếu trong tài liệu có hình ảnh nhưng không có chú thích, hãy thêm một mô tả ngắn cho hình ảnh đó bên trong thẻ <img></img>;
        nếu hình ảnh đã có chú thích, hãy đặt chú thích đó bên trong thẻ <img></img>.

        Dấu watermark nên được đặt trong thẻ <watermark></watermark>.
        Số trang nên được đặt trong thẻ <page_number></page_number>.
        Ví dụ: <page_number>14</page_number> hoặc <page_number>9/22</page_number>.

        Ưu tiên sử dụng ký hiệu ☐ và ☑ cho các ô kiểm (checkbox).

        QUAN TRỌNG: Luôn luôn trả lời bằng **Tiếng Việt**.
    """

    # user_prompt = "You are an helpful assistant"
    user_prompt = "Bạn là 1 trợ lý giúp trích xuất thông tin từ hình ảnh, hãy thực hiện đúng như yêu cầu đề ra."

    image = Image.open(image_path)
    messages = [
        {"role": "system", "content": user_prompt},
        {"role": "user", "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": system_prompt},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt")
    inputs = inputs.to(model.device)

    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]

    output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return output_text[0]


model, processor = load_ocr_model()

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.74s/it]


In [ ]:
image_path = "private-test-output\\Public048\\images\\test1.png"
result = ocr_page_with_nanonets_s(image_path, model, processor, max_new_tokens=1000)
print(result)

VIETTEL AI RACE
NHẬN ĐIỆN VỊ TRÌ BIỂN SỐ XE MÁY VIỆT NAM

TD048
Lần ban hành: 1

1. Lời mở đầu
Bài toán nhận diện biển số xe Việt Nam là một bài toán khó khăn của một dự án phát triển dựa trên các phương pháp xử lý ảnh truyền thống và cả những kỹ thuật sử dụng Deep Learning. Trong bài toán này tôi chỉ phát triển bài toán nhận diện biển số dãy tranh thiết toán YOLO-TinyV4 với mục đích:
*   **Hương vị**: Hướng dẫn chuẩn bị dữ liệu cho bài toán Object Detection.
*   **Hương vị**: Hướng dẫn huấn luyện YOLO-TinyV4 dùng darknet trên Google Colab.

2. Chuẩn bị dữ liệu
2.1 Đánh giá bộ dữ liệu
Trong bài viết tôi sử dụng bộ dữ liệu biển số xe máy Việt Nam chín 1750 ảnh, bạn đọc có thể tải tại đây:

<img>Ảnh 1</img>
Hình 1: Ảnh biển số trong bộ dữ liệu
*   **Định nghĩa**: Bộ dữ liệu được trong bộ dữ liệu được chụp từ một camera tại vị trí kiểm soát xe ra vào trung tâm. Dữ liệu.
*   **Kích thước**: Các biển số xe không có sự đa dạng, do khoảng cách từ camera đến biển số xe thấp xỉ nắp nhau giữa cá

In [10]:
with open("nanonet1.md", "w", encoding="utf-8") as fp:
    fp.write(result)

### Docling VLM

In [2]:
import os
print(os.getcwd())
input_doc_path = 'private-test-input\\Public048.pdf'
os.path.exists(input_doc_path)

d:\VIETTEL_RACE\qa-mcq-rag\qa-mcq-rag-main\app


True

In [12]:

import logging
from collections.abc import Iterable
from pathlib import Path
from docling_core.types.doc.document import DoclingDocument, NodeItem, TextItem
from docling_core.types.doc.labels import DocItemLabel
from docling.datamodel.base_models import InputFormat, ItemAndImageEnrichmentElement
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.models.base_model import BaseItemAndImageEnrichmentModel
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline


class ExampleFormulaUnderstandingPipelineOptions(PdfPipelineOptions):
    do_formula_understanding: bool = True


# A new enrichment model using both the document element and its image as input
class ExampleFormulaUnderstandingEnrichmentModel(BaseItemAndImageEnrichmentModel):
    images_scale = 2.6

    def __init__(self, enabled: bool):
        self.enabled = enabled

    def is_processable(self, doc: DoclingDocument, element: NodeItem) -> bool:
        return (
            self.enabled
            and isinstance(element, TextItem)
            and element.label == DocItemLabel.FORMULA
        )

    def __call__(
        self,
        doc: DoclingDocument,
        element_batch: Iterable[ItemAndImageEnrichmentElement],
    ) -> Iterable[NodeItem]:
        if not self.enabled:
            return

        for enrich_element in element_batch:
            # Opens a window for each cropped formula image; comment this out when
            # running headless or processing many items to avoid blocking spam.
            enrich_element.image.show()

            yield enrich_element.item


# How the pipeline can be extended.
class ExampleFormulaUnderstandingPipeline(StandardPdfPipeline):
    def __init__(self, pipeline_options: ExampleFormulaUnderstandingPipelineOptions):
        super().__init__(pipeline_options)
        self.pipeline_options: ExampleFormulaUnderstandingPipelineOptions

        self.enrichment_pipe = [
            ExampleFormulaUnderstandingEnrichmentModel(
                enabled=self.pipeline_options.do_formula_understanding
            )
        ]

        if self.pipeline_options.do_formula_understanding:
            self.keep_backend = True

    @classmethod
    def get_default_options(cls) -> ExampleFormulaUnderstandingPipelineOptions:
        return ExampleFormulaUnderstandingPipelineOptions()


# Example main. In the final version, we simply have to set do_formula_understanding to true.
def main():
    logging.basicConfig(level=logging.INFO)

    # data_folder =  Path.cwd()
    input_doc_path = "D:\\VIETTEL_RACE\\qa-mcq-rag\\qa-mcq-rag-main\\app\\private-test-input\\Public048.pdf"
    print(input_doc_path,':',os.path.exists(input_doc_path))

    pipeline_options = ExampleFormulaUnderstandingPipelineOptions()
    pipeline_options.do_formula_understanding = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_cls=ExampleFormulaUnderstandingPipeline,
                pipeline_options=pipeline_options,
            )
        }
    )
    result = doc_converter.convert(input_doc_path)
    print(result)

if __name__ == "__main__":
    main()

2025-10-18 17:46:45,489 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-18 17:46:45,493 - INFO - Going to convert document batch...
2025-10-18 17:46:45,494 - INFO - Initializing pipeline for ExampleFormulaUnderstandingPipeline with options hash edbae4e62284fcf3697fc850113a5765
2025-10-18 17:46:45,540 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-10-18 17:46:45,550 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-18 17:46:45,560 [RapidOCR] download_file.py:60: File exists and is valid: D:\Utilities\miniconda\envs\rag\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-10-18 17:46:45,560 [RapidOCR] main.py:53: Using D:\Utilities\miniconda\envs\rag\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-10-18 17:46:45,646 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-18 17:46:45,650 [RapidOCR] download_file.py:60: File exists and is valid: D:\Utilities\miniconda\envs\rag\Lib\site-packages\ra

D:\VIETTEL_RACE\qa-mcq-rag\qa-mcq-rag-main\app\private-test-input\Public048.pdf : True


[INFO] 2025-10-18 17:46:45,699 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-18 17:46:45,761 [RapidOCR] download_file.py:60: File exists and is valid: D:\Utilities\miniconda\envs\rag\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
[INFO] 2025-10-18 17:46:45,763 [RapidOCR] main.py:53: Using D:\Utilities\miniconda\envs\rag\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
2025-10-18 17:46:45,941 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-10-18 17:46:45,943 - INFO - Accelerator device: 'cuda:0'
2025-10-18 17:46:48,576 - INFO - Accelerator device: 'cuda:0'
2025-10-18 17:46:48,949 - INFO - Processing document Public048.pdf
2025-10-18 17:46:49,769 - WARNING - RapidOCR returned empty result!
2025-10-18 17:46:54,963 - WARNING - RapidOCR returned empty result!
2025-10-18 17:47:04,235 - WARNING - RapidOCR returned empty result!
2025-10-18 17:47:11,473 - WARNING - RapidOCR returned empty result!
2025-10-18 17:47:15,611 - WARNING

input=InputDocument(file=PureWindowsPath('Public048.pdf'), document_hash='3558f732c34abbef8b68527fcf64cf001feabea3d08e97d942e0a2a4682a93d5', valid=True, limits=DocumentLimits(max_num_pages=9223372036854775807, max_file_size=9223372036854775807, page_range=(1, 9223372036854775807)), format=<InputFormat.PDF: 'pdf'>, filesize=939744, page_count=6) status=<ConversionStatus.SUCCESS: 'success'> errors=[] pages=[Page(page_no=0, size=Size(width=595.3200073242188, height=841.9200439453125), predictions=PagePredictions(layout=LayoutPrediction(clusters=[Cluster(id=25, label=<DocItemLabel.SECTION_HEADER: 'section_header'>, bbox=BoundingBox(l=209.69, t=68.26599999999996, r=325.099, b=79.16699999999992, coord_origin=<CoordOrigin.TOPLEFT: 'TOPLEFT'>), confidence=0.690662682056427, cells=[PdfTextCell(index=1, rgba=ColorRGBA(r=0, g=0, b=0, a=255), rect=BoundingRectangle(r_x0=209.69, r_y0=79.16699999999992, r_x1=325.099, r_y1=79.16699999999992, r_x2=325.099, r_y2=68.26599999999996, r_x3=209.69, r_y3=68.